## SQL 과제 #3
1. 아래 2개의 테이블을 생성하고, 주어진 문제에 대한 sql문장을 작성하시오.
- 제출물: hw03.sql, hw03.ipynb 또는 hw03.py
- 데이터베이스 이름: shoppingmall
- 테이블 이름: user_table, buy_table

In [1]:
import pymysql

conn=pymysql.connect(host='localhost', user='root', password='0813', db='shoppingmall', charset='utf8')
cur=conn.cursor()


sql1="""
insert into user_table (userID, userName, birthYear, addr, mobile1, mobile2, height, mDate)
values (%s, %s, %s, %s, %s, %s, %s, %s)
"""
data1=(
    ('KHD','강호동',1970,'경북','011','22222222',182,'2007-07-07'),
    ('KJD','김제동',1974,'경남',None,None,173,'2013-03-03'),
    ('KKJ','김국진',1965,'서울','019','33333333',171,'2009-09-09'),
    ('KYM','김용만',1967,'서울','010','44444444',177,'2015-05-05'),
    ('LHJ','이휘재',1972,'경기','011','88888888',180,'2006-04-04'),
    ('LKK','이경규',1960,'경남','018','99999999',170,'2004-12-12'),
    ('NHS','남희석',1971,'충남','016','66666666',180,'2017-04-04'),
    ('PSH','박수홍',1970,'서울','010','00000000',183,'2012-05-05'),
    ('SDY','신동엽',1971,'경기',None,None,176,'2008-10-10'),
    ('YJS','유재석',1972,'서울','010','11111111',178,'2008-08-08'),
)

cur.executemany(sql1,data1)
conn.commit()

In [2]:
sql2="""
insert into buy_table (num, userID, prodName, groupName, price, amount)
values (%s, %s, %s, %s, %s, %s)
"""
data2=(
    (None,'KHD','운동화',None,30,2),
    (None,'KHD','노트북','전자',1000,1),
    (None,'KYM','모니터','전자',200,1),
    (None,'PSH','모니터','전자',200,5),
    (None,'KHD','청바지','의류',50,3),
    (None,'PSH','메모리','전자',80,10),
    (None,'KJD','책','서적',15,5),
    (None,'LHJ','책','서적',15,2),
    (None,'LHJ','청바지','의류',50,1),
    (None,'PSH','운동화',None,30,2),
    (None,'LHJ','책','서적',15,1),
    (None,'PSH','운동화',None,30,2)
)

cur.executemany(sql2,data2)
conn.commit()

2. 두 테이블을 내부 조인(buy_table.useID와 user_table.userID)한 다음, 아래의 결과와 같이
출력이 되도록 SQL 쿼리를 작성하시오.

1) 내부 조인한 결과에 ‘연락처’ 컬럼 추가

In [1]:
import pymysql

conn=pymysql.connect(host='localhost', user='root', password='0813', db='shoppingmall', charset='utf8')
cur=conn.cursor()

In [2]:
import pandas as pd

sql3="""
select ut.userName, bt.prodName, ut.addr, concat(ut.mobile1,ut.mobile2) as '연락처'
from user_table as ut
inner join buy_table as bt 
on bt.userID=ut.userID
"""
cur.execute(sql3)
rows=cur.fetchall()
DF1=pd.DataFrame(rows)
DF1.columns=['userName','prodName','addr','연락처']
DF1

,userName,prodName,addr,연락처
0,강호동,운동화,경북,01122222222
1,강호동,노트북,경북,01122222222
2,김용만,모니터,서울,01044444444
3,박수홍,모니터,서울,01000000000
4,강호동,청바지,경북,01122222222
5,박수홍,메모리,서울,01000000000
6,김제동,책,경남,None
7,이휘재,책,경기,01188888888
8,이휘재,청바지,경기,01188888888
9,박수홍,운동화,서울,01000000000


2) userID가 KYM인 사람이 구매한 물건과 회원 정보 출력

In [3]:
sql4="""
select ut.userID, ut.userName, bt.prodName, ut.addr, concat(ut.mobile1,ut.mobile2)
from user_table as ut
inner join buy_table as bt
on bt.userID=ut.userID
where ut.userID='KYM'
"""
cur.execute(sql4)
rows=cur.fetchall()
DF2=pd.DataFrame(rows)
DF2.columns=['userID','userName','prodName','addr','연락처']
DF2

,userID,userName,prodName,addr,연락처
0,KYM,김용만,모니터,서울,01044444444


3) 전체 회원이 구매한 목록을 회원 아이디 순으로 정렬

In [4]:
sql5="""
select ut.userID, ut.userName, bt.prodName, ut.addr, concat(ut.mobile1,ut.mobile2) as '연락처'
from user_table as ut
inner join buy_table as bt 
on bt.userID=ut.userID
order by ut.userID
"""
cur.execute(sql5)
rows=cur.fetchall()
DF3=pd.DataFrame(rows)
DF3.columns=['userID','userName','prodName','addr','연락처']
DF3

,userID,userName,prodName,addr,연락처
0,KHD,강호동,운동화,경북,01122222222
1,KHD,강호동,노트북,경북,01122222222
2,KHD,강호동,청바지,경북,01122222222
3,KJD,김제동,책,경남,None
4,KYM,김용만,모니터,서울,01044444444
5,LHJ,이휘재,책,경기,01188888888
6,LHJ,이휘재,청바지,경기,01188888888
7,LHJ,이휘재,책,경기,01188888888
8,PSH,박수홍,모니터,서울,01000000000
9,PSH,박수홍,메모리,서울,01000000000


4) 쇼핑몰에서 한 번이라도 구매한 기록이 있는 회원 정보를 회원 아이디 순으로 출력
(distinct 사용)

In [5]:
sql6="""
select distinct ut.userID, ut.userName, ut.addr
from user_table as ut
inner join buy_table as bt 
on bt.userID=ut.userID
"""
cur.execute(sql6)
rows=cur.fetchall()
DF4=pd.DataFrame(rows)
DF4.columns=['userID','userName','addr']
DF4

,userID,userName,addr
0,KHD,강호동,경북
1,KJD,김제동,경남
2,KYM,김용만,서울
3,LHJ,이휘재,경기
4,PSH,박수홍,서울


5) 쇼핑몰 회원 중에서 주소가 경북과 경남인 회원 정보를 회원 아이디 순으로 출력

In [6]:
sql7="""
select ut.userID, ut.userName, ut.addr, concat(ut.mobile1,ut.mobile2) as '연락처'
from user_table as ut
inner join buy_table as bt
on bt.userID=ut.userID
where addr in ('경북','경남')
order by ut.userID
"""
cur.execute(sql7)
rows=cur.fetchall()
DF5=pd.DataFrame(rows)
DF5.columns=['userID','userName','addr','연락처']
DF5

,userID,userName,addr,연락처
0,KHD,강호동,경북,01122222222
1,KHD,강호동,경북,01122222222
2,KHD,강호동,경북,01122222222
3,KJD,김제동,경남,None


In [7]:
cur.close()
conn.close()